# Упрощение текста при помощи T5

Задание: дообучить модель (модификацию T5) для упрощения русскоязычных текстов.

Данные: отрывок из корпуса RuAdapt.

In [ ]:
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

В этом туториале мы будем по минимуму пользоваться функционалом библиотеки Transformers и обучим модель практически так же, как сделали бы это в pytorch.

Для начала загрузим данные.

In [ ]:
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ML_training_data/ruadapt_enc.csv')

In [ ]:
train_df, val_df = train_test_split(df)

In [ ]:
train_df

,source,target
5081,Основа его собрания – фонд документов по истор...,Основа его собрания - фонд документов по истор...
964,Дачи строились на средства членов этих коопера...,Дачи строились на средства членов этих коопера...
4773,Под Москвой был сорван план молниеносной войны...,Под Москвой был сорван план молниеносной войны...
3243,С ХVII в. началось производство тульских пряни...,С ХVII в. началось производство тульских пряни...
4120,Царствование Николая II началось с трагедии 18...,Царствование Николая II началось с трагедии 18...
...,...,...
5623,"Франка (1917 г.), «О России» и др.","Франка (1917 г.), «О России» и др."
1660,Тимирязев (от тюркского тимер – то есть «желез...,Тимирязев (от тюркского тимер – то есть «желез...
3969,Баба-Яга упоминается А.С.,Бабу-Ягу упоминает А.С.
2330,Добронравова «Прощание с Братском» и «Девчонки...,Добронравова «Прощание с Братском» и «Девчонки...


In [ ]:
source_train = [i.strip() for i in train_df['source'].tolist()]
target_train = [i.strip() for i in train_df['target'].tolist()]

In [ ]:
lines = list(zip(source_train, target_train))

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

Backbone model - это та модель, которую мы дообучаем. Мы возьмем дистиллированную версию модели T5 (в ней остались только русские и немного английских векторов), предобученную для перефразирования, и попробуем обучить ее еще и упрощать тексты.

In [ ]:
backbone_model = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(backbone_model)
tokenizer = T5Tokenizer.from_pretrained(backbone_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Посылаем модель на gpu и определяем стратегию оптимизации.

In [ ]:
model.cuda();
optimizer = torch.optim.Adam(params=[p for p in model.parameters() if p.requires_grad], lr=1e-5)

Мы будем обучаться следующим образом:


1.   Брать одну случайную пару "сложное предложение, простое предложение" из датасета;
2.   Токенизировать ее предобученным токенайзером модели;
3.   Делать forward и backward pass;
4.   То же самое повторим еще iterations раз.



In [ ]:
import random

In [ ]:
def get_batch(sentence):
    # наши батчи - это просто пары предложений
    return sentence[0], sentence[1]

Посмотрим, что происходит при токенизации:

In [ ]:
test_tokens = tokenizer('Она знает язык животных и растений. ')

Токенайзер выдает для каждого токена его id из словаря, а также attention_mask:

In [ ]:
test_tokens

{'input_ids': [983, 558, 2419, 1167, 4217, 2829, 1074, 259, 279, 11188, 543, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Чтобы посмотреть, на какие именно subword-токены токенайзер делит предложение, можно воспользоваться функцией tokenize:

In [ ]:
tokenizer.tokenize('Она знает язык животных и растений. ')

['▁О',
 'на',
 '▁зна',
 'ет',
 '▁язык',
 '▁живот',
 'ных',
 '▁',
 'и',
 '▁растени',
 'й',
 '.']

Attention mask означает, на какие токены нужно обратить внимание. Например, на padding tokens внимания обращать не нужно. Паддинг происходит, например, внутри батчей из множества предложений разных размеров, чтобы унифицировать их длины перед подачей в модель. Рассмотрим на примере двух предложений разной длины:

In [ ]:
padded_sequences = tokenizer(['Она знает язык животных и растений. ', 'Но не грибов'], padding=True)

Тут можно понять, что под номером 1 в словаре находится токен конца предложения, а под номером 0 - паддинг:

In [ ]:
padded_sequences["input_ids"]

[[983, 558, 2419, 1167, 4217, 2829, 1074, 259, 279, 11188, 543, 260, 1],
 [1894, 401, 259, 18195, 685, 1, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
padded_sequences["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]

Модель также выдает списки словарных id слов выходной последовательности. Чтобы превратить их в предложение, нужно воспользоваться декодированием:

In [ ]:
tokenizer.decode(padded_sequences["input_ids"][0])

'Она знает язык животных и растений.</s>'

In [ ]:
iterations = 10000

all_loss = 0

for i in range(iterations):
    xx, yy = get_batch(random.choice(lines))
    x = tokenizer(xx, return_tensors='pt', padding=True).to(model.device)
    y = tokenizer(yy, return_tensors='pt', padding=True).to(model.device)
    # если токен является паддингом, мы принудительно занижаем его значение, чтобы модель его не предсказывала
    y.input_ids[y.input_ids==0] = -100
    loss = model(
        input_ids=x.input_ids,
        attention_mask=x.attention_mask,
        labels=y.input_ids,
        decoder_attention_mask=y.attention_mask,
        # технический параметр, чтобы вернуть больше информации из модели. Может понадобиться для дебаггинга
        return_dict=True
    ).loss
    loss.backward()
    all_loss += loss.item()
    optimizer.step()
    optimizer.zero_grad()

    # progress report каждую тысячу эпох
    if (i > 0) and (i % 1000 == 0):
      print('Step: {0}, loss: {1}'.format(i, all_loss/i))

Step: 1000, loss: 0.34990730755450206
Step: 2000, loss: 0.33511261076992377


KeyboardInterrupt: 

Теперь мы можем оценить работу нашей модели.

Во-первых, при помощи вызова model.eval() даем ей понять, что больше она не тренируется и веса менять не нужно.

Во-вторых, напишем уже знакомую вам по мастер-классу функцию для упрощения.

In [ ]:
model.eval()

def simplify(text, beams=3, grams=5, do_sample=True, num_return_sequences=1):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    # делаем так, чтобы максимальная длина генерируемого предложения не сильно превышала длину оригинального
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, # генерируем упрощения для всех х в батче
                         encoder_no_repeat_ngram_size=grams, # все н-граммы такого размера, присутствовавшие в инпуте, не могут присутствовать в выдаче
                         num_beams=beams, # сколько "лучей" в beam_search (сколько последовательностей запоминаем)
                         max_length=max_size,
                         do_sample=do_sample, # если выставить False, будет greedy decoding
                         num_return_sequences=num_return_sequences # сколько возможных упрощений возвращать
                         )
    return [tokenizer.decode(o, skip_special_tokens=True) for o in out]

Скачаем метрику для оценки упрощения SARI и посмотрим, как наша модель упрощает предложения.

In [ ]:
! pip install git+https://github.com/feralvam/easse@master

  Cloning https://github.com/feralvam/easse (to revision master) to /tmp/pip-req-build-a9h4ixi2
  Running command git clone --filter=blob:none --quiet https://github.com/feralvam/easse /tmp/pip-req-build-a9h4ixi2
  Resolved https://github.com/feralvam/easse to commit 6a4352ec299ed03fda8ee45445ca43d9c7673e89
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-0b_famik/tseval_b0cd4288045f4b918c2649d617cb8422
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-0b_famik/tseval_b0cd4288045f4b918c2649d617cb8422
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit dea8863683ea5946fd50184883c9be7a7339e821
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from easse.sari import corpus_sari

Возьмем десять случайных предложений, которые наша модель еще не видела, упростим и оценим их.

In [ ]:
i = 0


while i < 10:
  row_id = random.randint(0, len(val_df)-1)
  current_orig = val_df['source'].iloc[row_id]
  current_out = simplify(current_orig)
  current_simple = val_df['target'].iloc[row_id]

  current_sari = corpus_sari(orig_sents=[current_orig], sys_sents=current_out, refs_sents=[[current_simple]])
  print('Original sentence: ', current_orig)
  print('Simplification: ', current_out)
  print('SARI: ', current_sari)
  print('*'*10)
  i += 1

Original sentence:  Припевом песни стало сочинённое ребенком четверостишие:
Simplification:  ['Припевом песня стало сочиненное ребенком четверостишее:']
SARI:  34.21296296296296
**********
Original sentence:  У некоторых из них нет даже личных имен, а есть только клички (Актёр, Барон). 
Simplification:  ['У некоторых - даже личные имена, а есть лишь клички (Актёра, Барона).']
SARI:  23.412430362894764
**********
Original sentence:  Караваев), «Снеговик-почтовик» (режиссёр Л.А. 
Simplification:  ['Караваев), Театр «Снеговик» (режиссер Л. А.']
SARI:  30.685055685055683
**********
Original sentence:  Кутузов – директор Сухопутного Шляхетского кадетского корпуса, одновременно в 1795 г. – командующий русскими войсками в Финляндии. 
Simplification:  ['Кутузов – директор «Шляхетского» Кадетского корпуса.']
SARI:  19.586750968329913
**********
Original sentence:  Верующие после гражданской регистрации брака совершают обряд венчания в церкви. 
Simplification:  ['Верующие после гражданского брак